# Week3_1 Assignment

## [BASIC](#Basic) 
- 토크나이징이 완료된 위키 백과 코퍼스를 다운받고 **단어 사전을 구축하는 함수를 구현**할 수 있다.
- `Skip-Gram` 방식의 학습 데이터 셋을 생성하는 **Dataset과 Dataloader 클래스를 구현**할 수 있다.
- **Negative Sampling** 함수를 구현할 수 있다. 


## [CHALLENGE](#Challenge)
- Skip-Gram을 학습 과정 튜토리얼을 따라하며, **Skip-Gram을 학습하는 클래스를 구현**할 수 있다. 


## [ADVANCED](#Advanced)
- Skip-Gram 방식으로 word embedding을 학습하는 **Word2Vec 클래스를 구현**하고 실제로 학습할 수 있다.
- 학습이 완료된 word embedding을 불러와 **Gensim 패키지를 사용해 유사한 단어**를 뽑을 수 있다. 

### Reference
- [Skip-Gram negative sampling 한국어 튜토리얼](https://wikidocs.net/69141)
    - (참고) 위 튜토리얼에서는 target word와 context word 페어의 레이블은 1로, target word와 negative sample word 페어의 레이블은 0이 되도록 학습 데이터를 구현해 binary classification을 구현한다. 하지만 우리는 word2vec 논문 방식을 그대로 따르기 위해 label을 생성하지 않고 대신 loss 함수를 변행해서 binary classification을 학습할 것이다. 

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np
import re
from typing import List, Dict
import random

In [ ]:
!pip install transformers -qq

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
# seed
seed = 7777
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla K80
cuda


## Basic

### 토크나이징이 완료된 위키 백과 코퍼스 다운로드 및 불용어 사전 크롤링
- 나의 구글 드라이브에 데이터를 다운받아 영구적으로 사용할 수 있도록 하자. 
    - [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)
- 다운받은 데이터는 토크나이징이 완료된 상태이지만 불용어를 포함하고 있다. 따라서 향후 불용어를 제거하기 위해 불용어 사전을 크롤링하자. 
    - [불용어 사전 출처](https://www.ranks.nl/stopwords/korean)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 다운로드
# !pip install gdown
# !gdown https://drive.google.com/u/0/uc?id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx

# import zipfile
# sample_zip = zipfile.ZipFile('/content/drive/MyDrive/원티드AIML/tokenized.zip')
# sample_zip.extractall('/content/drive/MyDrive/원티드AIML/')
# sample_zip.close()

In [ ]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# Korean stop words: 677


In [ ]:
stop_words[0]

'아'

### 단어 사전 구축 함수 구현 
- 문서 리스트를 입력 받아 사전을 생성하는 `make_vocab()` 함수를 구현하라.
- 함수 정의
    - 입력 매개변수
        - docs : 문서 리스트
        - min_count : 최소 단어 등장 빈도수 (단어 빈도가 `min_count` 미만인 단어는 사전에 포함하지 않음)
    - 조건
        - 문서 길이 제한
            - 단어 개수가 3개 이하인 문서는 처리하지 않음. (skip)
        - 사전에 포함되는 단어 빈도수 제한
            - 단어가 빈도가 `min_count` 미만은 단어는 사전에 포함하지 않음.
        - 불용어 제거 
            - 불용어 리스트에 포함된 단어는 제거 
    - 반환값 
        - word2count : 단어별 빈도 사전 (key: 단어, value: 등장 횟수)
        - wid2word : 단어별 인덱스(wid) 사전 (key: 단어 인덱스(int), value: 단어)
        - word2wid : 인덱스(wid)별 단어 사전 (key: 단어, value: 단어 인덱스(int))

ta
```
# 코퍼스 로드
_DATA_DIR="/content/drive/MyDrive/NLP강의/강의자료/tokenized"

with open(os.path.join(_DATA_DIR, "wiki_ko_mecab.txt")) as reader:
    docs = reader.readlines()
```

In [ ]:
docs = []
data = open('/content/drive/MyDrive/원티드AIML/tokenized/wiki_ko_mecab.txt', 'r', encoding='utf-8')
docs = data.readlines() # 전체 문장을 list에 저장하는 함수입니다.

In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,237


In [ ]:
# 문서 개수를 500개로 줄임
docs=random.sample(docs,500)

In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 500


In [ ]:
# 문서 내 숫자, 영어 대소문자, 특수문자를 제거 (re package 사용)
import re
docs = [re.sub('[^가-힣]', ' ', doc) for doc in docs]

내 풀이

In [ ]:
print(f"Check : {docs[0][:1000]}")

Check : 남모 공주            는 신라 의 공주   왕족 으로 법흥왕 과 보과 공주 부여 씨 의 딸 이 며 백제 동성왕 의 외손녀 였 다   경쟁자 인 준정 과 함께 신라 의 초대 여성 원화   화랑   였 다   그 가 준정 에게 암살 당한 것 을 계기 로 화랑 은 여성 이 아닌 남성 미소년 으로 선발 하 게 되 었 다   신라 진흥왕 에게 는 사촌 누나 이 자 이모 가 된다   신라 의 청소년 조직 이 었 던 화랑도 는 처음 에 는 남모   준정 두 미녀 를 뽑 아 이 를   원화 라 했으며 이 들 주위 에 는       여 명 의 무리 를 따르 게 하 였 다   그러나 준정 과 남모 는 서로 최고 가 되 고자 시기 하 였 다   준정 은 박영실 을 섬겼 는데   지소태후 는 자신 의 두 번 째 남편 이 기 도 한 그 를 싫어해서 준정 의 원화 를 없애 고 낭도 가 부족 한 남모 에게 위화랑 의 낭도 를 더 해 주 었 다   그 뒤 남모 는 준정 의 초대 로 그 의 집 에 갔 다가 억지로 권하 는 술 을 받아마시 고 취한 뒤 준정 에 의해 강물 에 던져져 살해 되 었 다   이 일 이 발각 돼 준 정도 사형 에 처해지 고 나라 에서 는 귀족 출신 의 잘 생기 고 품행 이 곧 은 남자 를 뽑 아 곱 게 단장 한 후 이 를 화랑 이 라 칭하 고 받들 게 하 였 다     부왕 신라 제     대 국왕 법흥왕 모후 보과 공주 부여 씨                   공주 남모 공주 외조부 백제 제     대 국왕 동성왕 외조모 신라 이찬 비지 의 딸   화랑전사 마루                  년   배우   박효빈   신라 법흥왕 백제 동성왕 준정 화랑 분류         년 죽음 분류   신라 의 왕녀 분류   신라 의 왕족 분류   화랑 분류   암살 된 사람 분류   독살 된 사람 분류   법흥왕 


모범 답안

In [ ]:
# 문서 내 숫자, 영어 대소문자, 특수문자를 제거
docs = [re.sub('[0-9a-zA-Z-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', doc) for doc in docs]
print(f"Check : {docs[0][:1000]}")

Check : 남모 공주  南 毛 公主  는 신라 의 공주  왕족 으로 법흥왕 과 보과 공주 부여 씨 의 딸 이 며 백제 동성왕 의 외손녀 였 다  경쟁자 인 준정 과 함께 신라 의 초대 여성 원화  화랑  였 다  그 가 준정 에게 암살 당한 것 을 계기 로 화랑 은 여성 이 아닌 남성 미소년 으로 선발 하 게 되 었 다  신라 진흥왕 에게 는 사촌 누나 이 자 이모 가 된다  신라 의 청소년 조직 이 었 던 화랑도 는 처음 에 는 남모  준정 두 미녀 를 뽑 아 이 를  원화 라 했으며 이 들 주위 에 는    여 명 의 무리 를 따르 게 하 였 다  그러나 준정 과 남모 는 서로 최고 가 되 고자 시기 하 였 다  준정 은 박영실 을 섬겼 는데  지소태후 는 자신 의 두 번 째 남편 이 기 도 한 그 를 싫어해서 준정 의 원화 를 없애 고 낭도 가 부족 한 남모 에게 위화랑 의 낭도 를 더 해 주 었 다  그 뒤 남모 는 준정 의 초대 로 그 의 집 에 갔 다가 억지로 권하 는 술 을 받아마시 고 취한 뒤 준정 에 의해 강물 에 던져져 살해 되 었 다  이 일 이 발각 돼 준 정도 사형 에 처해지 고 나라 에서 는 귀족 출신 의 잘 생기 고 품행 이 곧 은 남자 를 뽑 아 곱 게 단장 한 후 이 를 화랑 이 라 칭하 고 받들 게 하 였 다   부왕 신라 제   대 국왕 법흥왕 모후 보과 공주 부여 씨  宝 果 公主 扶餘 氏   공주 남모 공주 외조부 백제 제   대 국왕 동성왕 외조모 신라 이찬 비지 의 딸  화랑전사 마루        년  배우  박효빈  신라 법흥왕 백제 동성왕 준정 화랑 분류     년 죽음 분류  신라 의 왕녀 분류  신라 의 왕족 분류  화랑 분류  암살 된 사람 분류  독살 된 사람 분류  법흥왕



내 풀이

In [ ]:
from collections import Counter

def make_vocab(docs:List[str], min_count:int):
    """
    'docs'문서 리스트를 입력 받아 단어 사전을 생성.
    
    return 
        - word2count : 단어별 빈도 사전
        - wid2word : 단어별 인덱스(wid) 사전 
        - word2wid : 인덱스(wid)별 단어 사전
    """    
    word2count = Counter()
    word2id = dict()
    id2word = dict()
    
    for doc in tqdm(docs):
        word_list = doc.split()

        # 1. 문서 길이 제한
        # 2. 임시 딕셔너리(_word2count)에 단어별 등장 빈도 기록
        # 3. 불용어 제거

        # 불용어 제거
        wo_sw_list = [word for word in word_list if not word in stop_words]

        # 문서 길이 제한
        if len(wo_sw_list) < 4:
            continue

        # 임시 딕셔너리(_word2count)에 단어별 등장 빈도 기록
        _word2count = Counter(wo_sw_list)
        dict.update(word2count, _word2count)

    # 4. 토큰 최소 빈도를 만족하는 토큰만 사전에 추가
    word2count = {key:value for key, value in word2count.items() if value >= min_count}
    word2id = {key: idx for idx, (key, value) in enumerate(word2count.items())}
    id2word = {idx: word for word, idx in word2id.items()}
    return word2count, word2id, id2word

In [ ]:
make_vocab(docs, min_count=5)

100%|██████████| 500/500 [00:03<00:00, 147.15it/s]


({'남모': 6,
  '는': 14,
  '준정': 9,
  '화랑': 155,
  '었': 10,
  '주': 5,
  '생산': 6,
  '직경': 7,
  '톤': 7,
  '버전': 6,
  '너지': 14,
  '헝가리': 7,
  '선수': 14,
  '여자': 5,
  '올림픽': 9,
  '대포': 6,
  '파이어': 8,
  '구형': 5,
  '미사일': 5,
  '중국': 6,
  '홍': 19,
  '국수': 25,
  '쿠스': 10,
  '파우': 8,
  '문자열': 11,
  '시동': 6,
  '입력': 5,
  '사운드': 40,
  '가장': 6,
  '검사': 9,
  '캐시': 9,
  '호출': 7,
  '게임': 5,
  '섹션': 8,
  '프로세서': 13,
  '청안': 6,
  '선남': 5,
  '라마': 8,
  '일본': 5,
  '함대': 5,
  '윤인완': 7,
  '만들': 5,
  '기원전': 20,
  '종교': 5,
  '가톨릭교회': 21,
  '성': 9,
  '보': 10,
  '정부': 5,
  '루이': 5,
  '시녀': 5,
  '아라곤': 10,
  '유럽': 12,
  '왕': 5,
  '서': 7,
  '영혼': 31,
  '머리': 11,
  '읍': 6,
  '코지마': 6,
  '극장판': 6,
  '마사키': 31,
  '오기노': 5,
  '포맷': 6,
  '스톤': 23,
  '음반': 13,
  '차트': 6,
  '시몬스': 9,
  '이사벨라': 10,
  '콘스탄티누스': 12,
  '혼인': 22,
  '유지': 8,
  '대머리': 8,
  '우': 5,
  '아종': 33,
  '까': 12,
  '숲': 9,
  '필터': 24,
  '기름': 6,
  '청소': 10,
  '구매': 6,
  '기호': 6,
  '주포': 5,
  '램': 9,
  '묶': 8,
  '앙각': 5,
  '분당': 5,
  '커스': 5,
  '갖추': 11,
  

In [ ]:
word2count, word2id, id2word = make_vocab(docs, min_count=5)

100%|██████████| 500/500 [00:02<00:00, 190.39it/s]


In [ ]:
doc_len = sum(word2count.values()) # 문서 내 모든 단어의 개수 (단어별 등장 빈도의 총 합)
print(f"{doc_len:,}")

14,508


In [ ]:
print(f"# unique word : {len(word2id):,}")

# unique word : 1,267


모범 답안

In [ ]:
def make_vocab(docs:List[str], min_count:int):
    """
    'docs'문서 리스트를 입력 받아 단어 사전을 생성.
    
    return 
        - word2count : 단어별 빈도 사전
        - wid2word : 단어별 인덱스(wid) 사전 
        - word2wid : 인덱스(wid)별 단어 사전
    """

    word2count = dict()
    word2id = dict()
    id2word = dict()

    _word2count = dict() # 단어별 등장 빈도를 기록하기 위한 임시 딕셔너리 생성
    for doc in tqdm(docs):
        word_list = doc.split()
        # 조건 1. 문서 길이 제한
        if len(word_list)>3:
            for word in word_list:
                # 조건 2. 불용어 제거
                if word in stop_words:
                    continue
                try:
                    _word2count[word]+=1
                except KeyError:
                    _word2count[word]=1

    # 조건 3. 토큰 최소 빈도를 만족하는 토큰만 사전(word2count)에 추가
    idx=0
    for w,c in _word2count.items():
        if c<min_count:
            continue
        word2count[w] = c
        word2id[w] = idx
        id2word[idx] = w
        idx+=1

    
    return word2count, word2id, id2word

In [ ]:
word2count, word2id, id2word = make_vocab(docs, min_count=5)

100%|██████████| 500/500 [00:02<00:00, 185.87it/s]


In [ ]:
doc_len = sum(word2count.values()) # 문서 내 모든 단어의 개수 (unique한 단어 개수 * 단어 등장 빈도)
print(f"{doc_len:,}")

164,843


In [ ]:
print(f"# unique word : {len(word2id):,}")

# unique word : 5,882


### Dataset 클래스 구현
- Skip-Gram 방식의 학습 데이터 셋(`Tuple(target_word, context_word)`)을 생성하는 `CustomDataset` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - docs: 문서 리스트
        - word2id: 단어별 인덱스(wid) 사전
        - window_size: Skip-Gram의 윈도우 사이즈
    - 메소드
        - `make_pair()`
            - 문서를 단어로 쪼개고, 사전에 존재하는 단어들만 단어 인덱스로 변경
            - Skip-gram 방식의 `(target_word, context_word)` 페어(tuple)들을 `pairs` 리스트에 담아 반환
        - `__len__()`
            - `pairs` 리스트의 개수 반환
        - `__getitem__(index)`
            - `pairs` 리스트를 인덱싱
    - 주의 사항
        - `nn.Module`를 부모 클래스로 상속 받음 


In [ ]:
from nltk.util import skipgrams
import itertools

내 풀이

In [ ]:
class CustomDataset(Dataset):
    """
    문서 리스트를 받아 skip-gram 방식의 (target_word, context_word) 데이터 셋을 생성
    """
    def __init__(self, docs:List[str], word2id:Dict[str,int], window_size:int=5):
        self.docs = docs
        self.word2id = word2id
        self.window_size = window_size
        self.pairs = self.make_pair()

    def make_pair(self):
        """
        (target, context) 형식의 Skip-gram pair 데이터 셋 생성 
        """
        
        # 문서 쪼개고 인덱스 변경
        pairs = []
        processed_docs = []
        for doc in self.docs:
            split_doc = doc.split(' ')
            processed_doc = []
            for word in split_doc:
                if word in word2id.keys():
                    processed_doc.append(self.word2id[word])
            processed_docs.append(processed_doc)

        pairs = []
        for d in processed_docs:
            skipgram = list(skipgrams(d, 2, self.window_size))
            for i in skipgram:
                pairs.append(i)
        return pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx][0], self.pairs[idx][1]

모범 답안

In [ ]:
class CustomDataset(Dataset):
    """
    문서 리스트를 받아 skip-gram 방식의 (target_word, context_word) 데이터 셋을 생성
    """
    def __init__(self, docs:List[str], word2id:Dict[str,int], window_size:int=5):
        self.docs = docs
        self.word2id = word2id
        self.window_size = window_size
        self.pairs = self.make_pair()
    
    def make_pair(self):
        """
        (target, context) 형식의 Skip-gram pair 데이터 셋 생성 
        """
        pairs = []
        for doc in tqdm(self.docs):
            word_ids = [] # 문서 내 (사전에 존재하는) 단어의 인덱스를 저장하기 위한 리스트 (=학습 데이터)
            for word in doc.split():
                # 1. 사전(word2id)에 존재하는 단어만 단어 인덱스(wid)로 변경해 학습 데이터에 추가
                if self.word2id.get(word):
                    word_ids.append(self.word2id.get(word))
                else:
                    continue

            # 2. 학습 데이터 구축
            for i, u in enumerate(word_ids):
                for j in range(self.window_size):
                    if i-1-j>=0: # target_word 기준 왼쪽 방향의 context_word들을 추가
                        pairs.append((u, word_ids[i-1-j]))
                    if i+1+j<len(word_ids): # target_word 기준 오른쪽 방향의 context_word들을 추가
                        pairs.append((u, word_ids[i+1+j]))
        return pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx]

In [ ]:
dataset = CustomDataset(docs, word2id, window_size=5)

In [ ]:
len(dataset)

166048

In [ ]:
dataset[0]

(0, 1)

In [ ]:
# verify (target word, context word)
for i, pair in enumerate(dataset):
    if i==100:
        break
    print(f"({id2word[pair[0]]}, {id2word[pair[1]]})")

(남모, 는)
(남모, 준정)
(남모, 화랑)
(남모, 준정)
(남모, 화랑)
(남모, 었)
(는, 준정)
(는, 화랑)
(는, 준정)
(는, 화랑)
(는, 었)
(는, 는)
(준정, 화랑)
(준정, 준정)
(준정, 화랑)
(준정, 었)
(준정, 는)
(준정, 었)
(화랑, 준정)
(화랑, 화랑)
(화랑, 었)
(화랑, 는)
(화랑, 었)
(화랑, 는)
(준정, 화랑)
(준정, 었)
(준정, 는)
(준정, 었)
(준정, 는)
(준정, 는)
(화랑, 었)
(화랑, 는)
(화랑, 었)
(화랑, 는)
(화랑, 는)
(화랑, 남모)
(었, 는)
(었, 었)
(었, 는)
(었, 는)
(었, 남모)
(었, 준정)
(는, 었)
(는, 는)
(는, 는)
(는, 남모)
(는, 준정)
(는, 는)
(었, 는)
(었, 는)
(었, 남모)
(었, 준정)
(었, 는)
(었, 준정)
(는, 는)
(는, 남모)
(는, 준정)
(는, 는)
(는, 준정)
(는, 남모)
(는, 남모)
(는, 준정)
(는, 는)
(는, 준정)
(는, 남모)
(는, 는)
(남모, 준정)
(남모, 는)
(남모, 준정)
(남모, 남모)
(남모, 는)
(남모, 준정)
(준정, 는)
(준정, 준정)
(준정, 남모)
(준정, 는)
(준정, 준정)
(준정, 는)
(는, 준정)
(는, 남모)
(는, 는)
(는, 준정)
(는, 는)
(는, 준정)
(준정, 남모)
(준정, 는)
(준정, 준정)
(준정, 는)
(준정, 준정)
(준정, 남모)
(남모, 는)
(남모, 준정)
(남모, 는)
(남모, 준정)
(남모, 남모)
(남모, 주)
(는, 준정)
(는, 는)
(는, 준정)
(는, 남모)


### 위에서 생성한 `dataset`으로 DataLoader  객체 생성
- `DataLoader` 클래스로 `train_dataloader`객체를 생성하라. 
    - 생성자 매개변수와 값
        - dataset = 위에서 생성한 dataset
        - batch_size = 64
        - shuffle = True

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
len(train_dataloader)

2595

### Negative Sampling 함수 구현
- Skip-Gram은 복잡도를 줄이기 위한 방법으로 negative sampling을 사용한다. 
- `sample_table`이 다음과 같이 주어졌을 때, sample_table에서 랜덤으로 값을 뽑아 (batch_size, n_neg_sample) shape의 matrix를 반환하는 `get_neg_v_negative_sampling()`함수를 구현하라. 
- Sample Table은 negative distribution을 따른다. 
    - [negative distribution 설명](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#How-are-negative-samples-drawn?)
- 함수 정의
    - 입력 매개변수
        - batch_size : 배치 사이즈, matrix의 row 개수 
        - n_neg_sample : negative sample의 개수, matrix의 column 개수
    - 반환값 
        - neg_v : 추출된 negative sample (2차원의 리스트)


In [ ]:
unig_dist  = {'apple': 0.023, 'bee': 0.12, 'desk': 0.34, 'chair': 0.517}

sum(unig_dist.values())

1.0

In [ ]:
alpha      = 3 / 4

noise_dist = {key: val ** alpha for key, val in unig_dist.items()}
Z = sum(noise_dist.values())
noise_dist_normalized = {key: val / Z for key, val in noise_dist.items()}
noise_dist_normalized

{'apple': 0.044813853132981724,
 'bee': 0.15470428538870049,
 'chair': 0.4626305591982827,
 'desk': 0.33785130228003507}

In [ ]:
noise_dist_normalized.values()

dict_values([0.044813853132981724, 0.15470428538870049, 0.33785130228003507, 0.4626305591982827])

In [ ]:
np.random.choice(list(noise_dist_normalized.keys()), size=2, p=list(noise_dist_normalized.values()))


array(['chair', 'chair'], dtype='<U5')

In [ ]:
# negative sample을 추출할 sample table 생성 (해당 코드를 참고)
sample_table = []
sample_table_size = doc_len

# noise distribution 생성
alpha = 3/4
frequency_list = np.array(list(word2count.values())) ** alpha
Z = sum(frequency_list)
ratio = frequency_list/Z
negative_sample_dist = np.round(ratio*sample_table_size)

for wid, c in enumerate(negative_sample_dist):
    sample_table.extend([wid]*int(c))

내 풀이

In [ ]:
def get_neg_v_negative_sampling(batch_size:int, n_neg_sample:int):
    """
    위에서 정의한 sample_table에서 (batch_size, n_neg_sample) shape만큼 랜덤 추출해 "네거티브 샘플 메트릭스"를 생성
    np.random.choice() 함수 활용 (위에서 정의한 sample_table을 함수의 argument로 사용)
    """
    neg_v = []
    for _ in range(batch_size):
        neg_v.append(np.random.choice(negative_sample_dist, size=n_neg_sample, p=ratio))    
    return neg_v

In [ ]:
get_neg_v_negative_sampling(4, 5)

[array([13.,  8., 11.,  8., 26.]),
 array([11., 11.,  8., 13.,  8.]),
 array([37., 35., 10., 10.,  8.]),
 array([91.,  7.,  7., 10.,  7.])]

모범 답안


In [ ]:
def get_neg_v_negative_sampling(batch_size:int, n_neg_sample:int):
    """
    (batch_size, n_neg_sample) shape의 네거티브 샘플 메트릭스 생성
    """
    neg_v = np.random.choice(
        sample_table, size=(batch_size, n_neg_sample)
    ).tolist()
    return neg_v

In [ ]:
get_neg_v_negative_sampling(4, 5)

[[52, 5544, 62, 3648, 2568],
 [394, 5247, 480, 4551, 1700],
 [340, 272, 86, 4529, 2120],
 [2025, 1128, 4625, 1168, 122]]

## Challenge

### 미니 튜토리얼
- 아래 튜토리얼을 따라하며 Skip-Gram 모델의 `forward` 및 `loss` 연산 방식을 이해하자
- Reference
    - [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
    - [torch bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html)
    - [Skip-Gram negative sampling loss function 설명 영문 블로그](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#Derivation-of-Cost-Function-in-Negative-Sampling)
    - [Skip-Gram negative sampling loss function 설명 한글 블로그](https://reniew.github.io/22/)

In [ ]:
# hyper parameter example
emb_size = 30000 # vocab size
emb_dimension = 300 # word embedding 차원
n_neg_sample = 5
batch_size = 32

In [ ]:
# 1. Embedding Matrix와 Context Matrix를 생성
u_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)
v_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)

In [ ]:
# 2. wid(단어 인덱스)를 임의로 생성
pos_u = torch.randint(high = emb_size, size = (batch_size,))
pos_v = torch.randint(high = emb_size, size = (batch_size,))
neg_v = get_neg_v_negative_sampling(batch_size, n_neg_sample)
print(f"Target word idx : {pos_u} Pos context word idx : {pos_v} Neg context word idx : {neg_v}\n")

Target word idx : tensor([14572, 10481, 25327, 20031, 10027,  6139, 17569, 24028, 11394, 14033,
        12346, 25481, 19493, 26680,   942,   955,  5524,   867, 27904,  2491,
         2942,  9336, 24810, 19743, 25878, 25576,  8457, 20755, 23879, 23977,
        23030, 29186]) Pos context word idx : tensor([10906,  9884, 24171, 22064,  8590, 27028, 14188,  8228, 24948, 12864,
        16590,  4498, 10483, 17456, 25003,  7229,  3242, 22617,  1203, 26626,
        25998,  1121, 14825, 11618, 10707, 21143,  9198,  3756, 15791, 17739,
         1858,  3334]) Neg context word idx : [array([35.,  8.,  8., 30., 17.]), array([54., 14., 10., 19.,  8.]), array([10., 16.,  7., 54.,  9.]), array([12.,  9.,  8., 11.,  8.]), array([ 7., 13., 12.,  8., 19.]), array([12.,  8., 16.,  8.,  7.]), array([24., 10., 20., 19., 91.]), array([ 8., 10.,  8., 10., 31.]), array([11., 35., 13.,  8.,  8.]), array([67.,  9.,  7., 31.,  8.]), array([14., 13.,  8.,  8., 18.]), array([ 8., 11., 53., 26., 29.]), array([18.,  

In [ ]:
# 3. tensor로 변환
pos_u = Variable(torch.LongTensor(pos_u)).to(device)
pos_v = Variable(torch.LongTensor(pos_v)).to(device)
neg_v = Variable(torch.LongTensor(neg_v)).to(device)

In [ ]:
# 4. wid로 각각의 embedding matrix에서 word embedding 값을 가져오기
pos_u = u_embedding(pos_u)
pos_v = v_embedding(pos_v)
neg_v = v_embedding(neg_v)
print(f"shape of pos_u embedding : {pos_u.shape}\n shape of pos_v embedding : {pos_v.shape}\n shape of neg_v embedding : {neg_v.shape}")

shape of pos_u embedding : torch.Size([32, 300])
 shape of pos_v embedding : torch.Size([32, 300])
 shape of neg_v embedding : torch.Size([32, 5, 300])


In [ ]:
# 5. dot product 
pos_score = torch.mul(pos_u, pos_v) # 행렬 element-wise 곱
pos_score = torch.sum(pos_score, dim=1)
print(f"shape of pos logits : {pos_score.shape}\n")

neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze()
print(f"shape of logits : {neg_score.shape}")

shape of pos logits : torch.Size([32])

shape of logits : torch.Size([32, 5])


In [ ]:
# 6. loss 구하기
pos_score = F.logsigmoid(pos_score)
neg_score = F.logsigmoid(-1*neg_score) # negative의 logit은 minimize 하기 위해 -1 곱함
print(f"pos logits : {pos_score.sum()}")
print(f"neg logits : {neg_score.sum()}")
loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))
print(f"Loss : {loss}")

pos logits : -324.05657958984375
neg logits : -1160.510009765625
Loss : 1484.566650390625


### Skip-gram 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `SkipGram` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `device` : 연산 장치 종류
    - 생성자에서 생성해야할 변수 
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `u_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (target_word)
        - `v_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (context_word)
    - 메소드
        - `init_embedding()` (제공됨)
            - 엠베딩 메트릭스 값을 초기화
        - `forward()`
            - 위 튜토리얼과 같이 dot product를 수행한 후 score를 생성
            - loss를 반환 (loss 설명 추가)
        - `save_emedding()` (제공됨)
            - `u_embedding`의 단어 엠베딩 값을 단어 별로 파일에 저장
    - 주의 사항     
        - `nn.Module`를 부모 클래스로 상속 받음 

내 풀이

In [ ]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size:int, emb_dimension:int, device:str):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dimension = emb_dimension
        self.u_embedding = nn.Embedding(vocab_size, self.emb_dimension, sparse=True) .to(device)
        self.v_embedding = nn.Embedding(vocab_size, self.emb_dimension, sparse=True) .to(device)
        self.init_embedding()
    
    
    def init_embedding(self):
        """
        u_embedding과 v_embedding 메트릭스 값을 초기화
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embedding.weight.data.uniform_(-initrange, initrange)
        self.v_embedding.weight.data.uniform_(-0, 0)
    
    
    def forward(self, pos_u, pos_v, neg_v):
        """
        dot product를 수행한 후 score를 생성
        loss 반환
        """    
            
        # 각각의 embedding matrix에서 word embedding 값을 가져오기
        pos_u = self.u_embedding(pos_u)
        pos_v = self.v_embedding(pos_v)
        neg_v = self.v_embedding(neg_v)
        
        # dot product 
        pos_score = torch.sum(torch.mul(pos_u, pos_v), dim=1)
        neg_score = torch.sum(torch.bmm(neg_v, pos_u.unsqueeze(2)).squeeze(), dim=1)
        
        # loss 구하기
        loss = F.logsigmoid(pos_score).squeeze() + F.logsigmoid(-1*neg_score).squeeze()

        return -1*loss.sum()
    
    def save_embedding(self, id2word, file_name, use_cuda):
        """
        'file_name' 위치에 word와 word_embedding을 line-by로 저장
        파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
        """
        if use_cuda: # parameter를 gpu 메모리에서 cpu 메모리로 옮김
            embedding = self.u_embedding.weight.cpu().data.numpy()
        else:
            embedding = self.u_embedding.weight.data.numpy()

        with open(file_name, 'w') as writer:
            # 파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
            writer.write(f"{len(id2word)} {embedding.shape[-1]}\n")
            
            for wid, word in id2word.items():
                e = embedding[wid]
                e = " ".join([str(e_) for e_ in e])
                writer.write(f"{word} {e}\n")

모범 답안

In [ ]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size:int, emb_dimension:int, device:str):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dimension = emb_dimension
        self.u_embedding = nn.Embedding(vocab_size, emb_dimension, sparse=True).to(device)
        self.v_embedding = nn.Embedding(vocab_size, emb_dimension, sparse=True).to(device)
        self.init_embedding()
    
    
    def init_embedding(self):
        """
        u_embedding과 v_embedding 메트릭스 값을 초기화
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embedding.weight.data.uniform_(-initrange, initrange)
        self.v_embedding.weight.data.uniform_(-0, 0)
    
    
    def forward(self, pos_u, pos_v, neg_v):
        """
        ...
        """    
            
        pos_u = self.u_embedding(pos_u)
        pos_v = self.v_embedding(pos_v)
        neg_v = self.v_embedding(neg_v)
        
        pos_score = torch.mul(pos_u, pos_v)
        pos_score = torch.sum(pos_score, dim=1)
        pos_score = F.logsigmoid(pos_score)
        neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze()
        neg_score = F.logsigmoid(-1 * neg_score)
        
        loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))
        return loss
    
    def save_embedding(self, id2word, file_name, use_cuda):
        """
        'file_name' 위치에 word와 word_embedding을 line-by로 저장
        """
        if use_cuda: # parameter를 gpu 메모리에서 cpu 메모리로 옮김
            embedding = self.u_embedding.weight.cpu().data.numpy()
        else:
            embedding = self.u_embedding.weight.data.numpy()
        
        with open(file_name, 'w') as writer:
            # 파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
            writer.write(f"{len(id2word)} {embedding.shape[-1]}\n")
            
            for wid, word in id2word.items():
                e = embedding[wid]
                e = " ".join([str(e_) for e_ in e])
                writer.write(f"{word} {e}\n")
                

## Advanced

### Skip-Gram 방식의  Word2Vec 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `Word2Vec` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()`) 입력 매개 변수
        - `input_file` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `device` : 연상 장치 종류
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `min_count` : 사전에 추가될 단어의 최소 등장 빈도
    - 생성자에서 생성해야 할 변수 
        - `docs` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `word2count`, `word2id`, `id2word` : 위에서 구현한 `make_vocab()` 함수의 반환 값
        - `device` : 연산 장치 종류
        - `emb_size` : vocab의 (unique한) 단어 종류 
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `model` : `SkipGram` 클래스의 인스턴스
        - `optimizer` : `SGD` 클래스의 인스턴스
    - 메소드
        - `train()`
            - 입력 매개변수 
                - `train_dataloader`
            - Iteration 횟수만큼 input_file 학습 데이터를 학습한다. 매 epoch마다 for loop 돌면서 batch 단위 학습 데이터를 skip gram 모델에 학습함. 학습이 끝나면 word embedding을 output_file_name 파일에 저장.
- Reference
    - [Optimizer - SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

내 풀이

In [ ]:
class Word2Vec:
    def __init__(self, 
                input_file: List[str],
                output_file_name: str,
                 device: str,
                 emb_dimension=300,
                 batch_size = 64,
                 window_size=5,
                 n_neg_sample = 5,
                 iteration=1,
                 lr = 0.02,
                 min_count=5):
        self.docs = input_file
        self.output_file_name = output_file_name
        self.word2count, self.word2id, self.id2word = make_vocab(input_file, min_count=5)
        self.device = device
        self.emb_size = len(self.word2count)
        self.emb_dimension = emb_dimension
        self.batch_size = batch_size
        self.window_size = window_size
        self.n_neg_sample = n_neg_sample
        self.iteration = iteration
        self.lr = lr
        self.model = SkipGram(vocab_size=self.emb_size, emb_dimension=self.emb_dimension, device=self.device)
        self.optimizer = torch.optim.SGD(params=self.model.parameters(), lr=self.lr) # torch.optim.SGD 클래스 사용

        # train() 함수에서 만든 임베딩 결과 파일들을 저장할 폴더 생성 (os.makedirs 사용)
        os.makedirs(self.output_file_name, exist_ok=True)
        
    
    def train(self, train_dataloader):
        
        # lr 값을 조절하는 스케줄러 인스턴스 변수를 생성
        self.scheduler = get_linear_schedule_with_warmup(
            optimizer = self.optimizer,
            num_warmup_steps=0,
            num_training_steps=len(train_dataloader) * self.iteration
        )
        
        for epoch in range(self.iteration):
            
            print(f"*****Epoch {epoch} Train Start*****")
            print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
            total_loss, batch_loss, batch_step = 0,0,0

            for step, batch in enumerate(train_dataloader):
                batch_step+=1

                pos_u, pos_v = batch
                # negative data 생성
                neg_v = get_neg_v_negative_sampling(pos_u.shape[0], self.n_neg_sample)
                
                # 데이터를 tensor화 & device 설정
                pos_u = torch.LongTensor(pos_u).to(self.device)
                pos_v = torch.LongTensor(pos_v).to(self.device)
                neg_v = torch.LongTensor(neg_v).to(self.device)

                # model의 gradient 초기화
                self.model.zero_grad() 
                # optimizer의 gradient 초기화
                self.optimizer.zero_grad()

                # forward
                loss = self.model(pos_u, pos_v, neg_v)

                # loss 계산
                loss.backward()
                # optimizer 업데이트
                self.optimizer.step() 
                # scheduler 업데이트
                self.scheduler.step()

                batch_loss += loss.item()
                total_loss += loss.item()
                
                if (step%500 == 0) and (step!=0):
                    print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {self.optimizer.param_groups[0]['lr']:.4f}")
                    # 변수 초기화    
                    batch_loss, batch_step = 0,0
            
            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finished*****\n")
            
            print(f"*****Epoch {epoch} Saving Embedding...*****")
            self.model.save_embedding(self.id2word, os.path.join(self.output_file_name, f'w2v_{epoch}.txt'), True if 'cuda' in self.device.type else False)
            print(f"*****Epoch {epoch} Embedding Saved at {os.path.join(self.output_file_name, f'w2v_{epoch}.txt')}*****\n")

In [ ]:
output_file = os.path.join(".", "word2vec_wiki")
# Word2Vec 클래스의 인스턴스 생성
w2v = Word2Vec(docs, output_file, device, n_neg_sample=10, iteration=3)

# 학습 데이터 셋 및 데이터 로더 생성 (위에서 생성한 w2v의 attribute들을 argument에 적절히 넣기)
dataset = dataset
train_dataloader = train_dataloader
print(len(train_dataloader))

# 학습
w2v.train(train_dataloader)

100%|██████████| 500/500 [00:02<00:00, 187.14it/s]


2595
*****Epoch 0 Train Start*****
*****Epoch 0 Total Step 2595*****
Step: 500 Loss: 40.2254 lr: 0.0187
Step: 1000 Loss: 27.2442 lr: 0.0174
Step: 1500 Loss: 22.1698 lr: 0.0161
Step: 2000 Loss: 19.8511 lr: 0.0149
Step: 2500 Loss: 17.9831 lr: 0.0136
Epoch 0 Total Mean Loss : 25.2121
*****Epoch 0 Train Finished*****

*****Epoch 0 Saving Embedding...*****
*****Epoch 0 Embedding Saved at ./word2vec_wiki/w2v_0.txt*****

*****Epoch 1 Train Start*****
*****Epoch 1 Total Step 2595*****
Step: 500 Loss: 16.3507 lr: 0.0120
Step: 1000 Loss: 15.3007 lr: 0.0108
Step: 1500 Loss: 14.4759 lr: 0.0095
Step: 2000 Loss: 13.6116 lr: 0.0082
Step: 2500 Loss: 13.3432 lr: 0.0069
Epoch 1 Total Mean Loss : 14.5554
*****Epoch 1 Train Finished*****

*****Epoch 1 Saving Embedding...*****
*****Epoch 1 Embedding Saved at ./word2vec_wiki/w2v_1.txt*****

*****Epoch 2 Train Start*****
*****Epoch 2 Total Step 2595*****
Step: 500 Loss: 12.9155 lr: 0.0054
Step: 1000 Loss: 12.4151 lr: 0.0041
Step: 1500 Loss: 12.1056 lr: 0.002

모범 답안

In [ ]:
class Word2Vec:
    def __init__(self, 
                input_file: List[str],
                output_file_name: str,
                 device: str,
                 emb_dimension=300,
                 batch_size = 64,
                 window_size=5,
                 n_neg_sample = 5,
                 iteration=1,
                 lr = 0.02,
                 min_count=5):
        self.docs = input_file
        self.output_file_name = output_file_name
        self.word2count, self.word2id, self.id2word = make_vocab(self.docs, min_count=min_count)
        self.device = device
        self.emb_size = len(self.word2id)
        self.emb_dimension = emb_dimension
        self.batch_size =batch_size
        self.window_size = window_size
        self.n_neg_sample = n_neg_sample
        self.iteration = iteration
        self.lr = lr
        self.model = SkipGram(self.emb_size, self.emb_dimension, self.device)
        self.optimizer = SGD(
            self.model.parameters(),
            lr = self.lr
        )
        # 파일 저장할 폴더 생성
        os.makedirs(self.output_file_name, exist_ok=True)
        
    
    def train(self, train_dataloader):
        
        # lr 값을 조절하는 스케줄러 인스턴스 변수를 생성
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=0,
            num_training_steps=len(train_dataloader)*self.iteration
        )
        
        for epoch in range(self.iteration):
            
            print(f"*****Epoch {epoch} Train Start*****")
            print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
            total_loss, batch_loss, batch_step = 0,0,0

            for step, batch in enumerate(train_dataloader):
                batch_step+=1

                pos_u, pos_v = batch
                # negative data 생성
                # neg_v = get_neg_v_negative_sampling(self.batch_size, self.n_neg_sample)
                neg_v = get_neg_v_negative_sampling(pos_u.shape[0], self.n_neg_sample)
                
                # 데이터를 tensor화 & device 설정
                pos_u = Variable(torch.LongTensor(pos_u)).to(self.device)
                pos_v = Variable(torch.LongTensor(pos_v)).to(self.device)
                neg_v = Variable(torch.LongTensor(neg_v)).to(self.device)

                # gradient 초기화
                self.optimizer.zero_grad()
                self.model.zero_grad()

                # forward
                loss = self.model.forward(pos_u, pos_v, neg_v)

                # loss
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()

                batch_loss += loss.item()
                total_loss += loss.item()
                
                if (step%500 == 0) and (step!=0):
                    print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {self.optimizer.param_groups[0]['lr']:.4f}")
                    # 변수 초기화    
                    batch_loss, batch_step = 0,0
            
            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finished*****\n")
            
            print(f"*****Epoch {epoch} Saving Embedding...*****")
            self.model.save_embedding(self.id2word, os.path.join(self.output_file_name, f'w2v_{epoch}.txt'), True if 'cuda' in self.device.type else False)
            print(f"*****Epoch {epoch} Embedding Saved at {os.path.join(self.output_file_name, f'w2v_{epoch}.txt')}*****\n")
                    



In [ ]:
# Word2Vec 클래스의 인스턴스 생성
output_file = os.path.join(".", "word2vec_wiki")
w2v = Word2Vec(docs, output_file, device, n_neg_sample=10, iteration=3)

100%|██████████| 500/500 [00:01<00:00, 297.95it/s]


In [ ]:
# 학습 데이터 셋 및 데이터 로더 생성
dataset = CustomDataset(w2v.docs, w2v.word2id, w2v.window_size)
train_dataloader = DataLoader(dataset, w2v.batch_size)
len(train_dataloader)

100%|██████████| 500/500 [00:00<00:00, 956.28it/s] 


21482

In [ ]:
w2v.train(train_dataloader)

*****Epoch 0 Train Start*****
*****Epoch 0 Total Step 21482*****
Step: 500 Loss: 484.5451 lr: 0.0198
Step: 1000 Loss: 395.7239 lr: 0.0197
Step: 1500 Loss: 248.9553 lr: 0.0195
Step: 2000 Loss: 200.7881 lr: 0.0194
Step: 2500 Loss: 119.2248 lr: 0.0192
Step: 3000 Loss: 104.1140 lr: 0.0191
Step: 3500 Loss: 229.0389 lr: 0.0189
Step: 4000 Loss: 235.0788 lr: 0.0188
Step: 4500 Loss: 237.9144 lr: 0.0186
Step: 5000 Loss: 212.8852 lr: 0.0184
Step: 5500 Loss: 192.6870 lr: 0.0183
Step: 6000 Loss: 209.6250 lr: 0.0181
Step: 6500 Loss: 215.1007 lr: 0.0180
Step: 7000 Loss: 164.4239 lr: 0.0178
Step: 7500 Loss: 170.4262 lr: 0.0177
Step: 8000 Loss: 187.0456 lr: 0.0175
Step: 8500 Loss: 204.3405 lr: 0.0174
Step: 9000 Loss: 194.8780 lr: 0.0172
Step: 9500 Loss: 205.0695 lr: 0.0171
Step: 10000 Loss: 186.4944 lr: 0.0169
Step: 10500 Loss: 185.5174 lr: 0.0167
Step: 11000 Loss: 197.1707 lr: 0.0166
Step: 11500 Loss: 194.6444 lr: 0.0164
Step: 12000 Loss: 188.4481 lr: 0.0163
Step: 12500 Loss: 197.3222 lr: 0.0161
Step:

### 유사한 단어 확인
- 사전에 존재하는 단어들과 유사한 단어를 검색해보자. Gensim 패키지는 유사 단어 외에도 단어간의 유사도를 계산하는 여러 함수를 제공한다. 실험을 통해 word2vec의 한계점을 발견했다면 아래에 markdown으로 작성해보자. 
- [Gensim 패키지 document](https://radimrehurek.com/gensim/models/keyedvectors.html)

내 풀이

In [ ]:
import gensim

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('./word2vec_wiki/w2v_2.txt', binary=False)

In [ ]:
word_vectors.most_similar(positive=['여자', '준정'], negative=['화랑'])

[('너지', 0.9981141686439514),
 ('었', 0.9978345632553101),
 ('일본', 0.99715256690979),
 ('배구', 0.9971135258674622),
 ('버전', 0.9969197511672974),
 ('비아', 0.9968982934951782),
 ('게임', 0.9968740344047546),
 ('나미', 0.9968181848526001),
 ('기라티나', 0.9966198801994324),
 ('는', 0.9966039657592773)]

word2vec의 한계점은?
- vocab 사이즈가 커야지 많은 관계를 표현할 수 있다. vocab 사이즈가 작은 경우 완전 생뚱맞은 관계를 입력하면 이상한 값이 나온다.

- vocabulary의 사이즈가 크고 vocab 외의 단어를 표현하지 못한다. 데이터 전체에 대한 정보를 담기 어렵다. 데이터 전체에 대한 빈도를 반영이 되지 않는다.

모범 답안

In [ ]:
import gensim

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('./word2vec_wiki/w2v_1.txt', binary=False)

In [ ]:
word_vectors.most_similar(positive=['대통령'])

[('주한', 0.9916805624961853),
 ('광역시', 0.991572380065918),
 ('육군사관학교', 0.9912100434303284),
 ('보병', 0.9907797574996948),
 ('방송국', 0.9904221892356873),
 ('박물관', 0.9901329278945923),
 ('한양', 0.9900871515274048),
 ('알아인', 0.9899305105209351),
 ('당원', 0.98966383934021),
 ('개신교도', 0.9896210432052612)]

In [ ]:
word_vectors.most_similar(positive=['코난'])

[('막달레나', 0.9994158744812012),
 ('명탐정', 0.9993422627449036),
 ('재상', 0.9993184208869934),
 ('뉴캐슬', 0.9992764592170715),
 ('린든', 0.9992668628692627),
 ('줄넘기', 0.9992555975914001),
 ('공예', 0.9992364048957825),
 ('워즈', 0.9992086887359619),
 ('찹', 0.9991920590400696),
 ('배기', 0.9991424083709717)]